Illia Bannitsyn illiabann@student.agh.edu.pl

Patrycja Marchwica patmar@student.agh.edu.pl

# Zadanie 1 
### Przygotować swoje środowisko pracy (można korzystać z Google Colab)

# Import danych

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from keras import models
from keras import layers
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf
from tensorflow.compat.v1.keras import backend as K


#config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 48} , log_device_placement=True )
#sess = tf.compat.v1.Session(config=config) 
#K.set_session(sess)
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

## Funkcje

### Deskew

In [2]:
SZ=28
affine_flags = cv2.WARP_INVERSE_MAP|cv2.INTER_LINEAR

def deskew(img):
    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    skew = m['mu11']/m['mu02']
    M = np.float32([[1, skew, -0.5*SZ*skew], [0, 1, 0]])
    img = cv2.warpAffine(img,M,(SZ, SZ),flags=affine_flags)
    return img

### HOG (Histogram of Oriented Gradients)

In [3]:
def HOG(im):
    winSize = (20,20)
    blockSize = (10,10)
    blockStride = (5,5)
    cellSize = (10,10)
    nbins = 9
    derivAperture = 1
    winSigma = -1.
    histogramNormType = 0
    L2HysThreshold = 0.2
    gammaCorrection = 1
    nlevels = 64
    signedGradients = True

    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,histogramNormType,L2HysThreshold,gammaCorrection,nlevels, signedGradients)
    descriptor = np.ravel(hog.compute(im))
    
    return descriptor

### Tworzenie modelu dla sieci

In [4]:
def create_model():
  model = models.Sequential()
  model.add(layers.Dense(512, activation='relu'))
  model.add(layers.Dense(10, activation='softmax'))
  model.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  return model

### Grid Search - Walidacja krzyżowa

In [5]:
def SVMGridSearch(train_data, train_labels):
  parameters = {'kernel':('linear', 'rbf'), 'C': np.linspace(start = 0.001, stop = 2, num = 50)}
  svc = svm.SVC()
  clf = GridSearchCV(svc, parameters, scoring='accuracy', cv=StratifiedKFold())
  clf.fit(train_data, train_labels)
  best_params = clf.best_params_
  my_svc = clf.best_estimator_
  
  print("Best hiperparameters: ", best_params)
  
  return my_svc

In [6]:
def RandomForestGridSearch(train_data, train_labels):
  parameters = {
    'max_depth': [5, 8, 15, 25, 30],
    'min_samples_leaf': [1, 2, 5, 10],
    'min_samples_split': [2, 5, 10, 15, 100],
    'n_estimators': [20, 30, 50, 70, 100]
  }

  rf = RandomForestClassifier()
  clf = GridSearchCV(estimator = rf, param_grid = parameters, cv = 3, n_jobs = -1, verbose = 1)
  clf.fit(train_data, train_labels)
  best_params = clf.best_params_
  my_rf = clf.best_estimator_
  
  print("Best hiperparameters: ", best_params)

  return my_rf

In [7]:
def NeuralNetworkGridSrearch(train_data, train_labels):
  parameters = {
      'batch_size': [30, 50, 100],
      'epochs': [5, 10, 15]
      }
  kc = KerasClassifier(build_fn=create_model, verbose=0)
  clf = GridSearchCV(estimator=kc, param_grid=parameters, n_jobs=-1, cv=3)
  clf.fit(train_data, train_labels)
  best_params = clf.best_params_

  print("Best hiperparameters: ", best_params)

  return best_params

### Macierz pomyłek i raporty

In [8]:
def getConfMatrix(test_labels, pred_labels_deskewed, pred_labels_non_deskewed,pred_labels_raw_deskewed,pred_labels_shuffle):
  cm_deskewed = confusion_matrix(test_labels, pred_labels_deskewed)
  cm_non_deskewed = confusion_matrix(test_labels, pred_labels_non_deskewed)
  cm_raw_deskewed = confusion_matrix(test_labels, pred_labels_raw_deskewed)
  cm_shuffle = confusion_matrix(test_labels, pred_labels_shuffle)


  print("Macierz pomyłek - dane wyrównane:")
  print(cm_deskewed)
  print("Macierz pomyłek - dane niewyrównane:")
  print(cm_non_deskewed)
  print("Macierz pomyłek - dane surowe:")
  print(cm_raw_deskewed)
  print("Macierz pomyłek - dane zmieszane:")
  print(cm_shuffle)

In [9]:
def getReports(test_labels, pred_labels_deskewed, pred_labels_non_deskewed,pred_labels_raw_deskewed,pred_labels_shuffle):
  print("Raport - dane wyrównane:")
  print(classification_report(test_labels, pred_labels_deskewed))
  print("Raport - dane niewyrównane:")
  print(classification_report(test_labels, pred_labels_non_deskewed))
  print("Raport - dane surowe:")
  print(classification_report(test_labels, pred_labels_raw_deskewed))
  print("Raport - dane zmieszane:")
  print(classification_report(test_labels, pred_labels_shuffle))

In [10]:
def getAccuracy(test_labels, pred_labels_deskewed, pred_labels_non_deskewed,pred_labels_raw_deskewed,pred_labels_shuffle):
  print("Accuracy - dane wyrównane: " + str(accuracy_score(test_labels, pred_labels_deskewed)))
  print("Accuracy - dane niewyrównane: " + str(accuracy_score(test_labels, pred_labels_non_deskewed)))
  print("Accuracy - dane surowe: " + str(accuracy_score(test_labels, pred_labels_raw_deskewed)))
  print("Accuracy - dane zmieszane: " + str(accuracy_score(test_labels, pred_labels_shuffle)))

In [11]:
def single_result(test_labels,pred_labels,dane):
  cm = confusion_matrix(test_labels, pred_labels)
  print("Macierz pomyłek - "+dane+":")
  print(cm)
  print("Raport - "+dane+":")
  print(classification_report(test_labels, pred_labels))
  print("Accuracy -  "+dane+":" + str(accuracy_score(test_labels, pred_labels)))


## Preprocessing danych

### Surowe dane

In [12]:
train_raw = train_images.reshape(len(train_images), 28 * 28)
test_raw = test_images.reshape(len(test_images), 28 * 28)

###  Dane poddane wyrównaniu

In [13]:
train_deskewed = np.float32([deskew(im) for im in train_raw])
test_deskewed = np.float32([deskew(im) for im in test_raw])

train_deskewed = np.asarray(train_deskewed).reshape(-1,28*28)
test_deskewed = np.asarray(test_deskewed).reshape(-1,28*28)

In [14]:
hogdata_train = np.float32([HOG(im) for im in train_images]).reshape(-1,81)
hogdata_test = np.float32([HOG(im) for im in test_images]).reshape(-1,81)

hogdata_train_deskewed = np.float32([HOG(deskew(im)) for im in train_images]).reshape(-1,81)
hogdata_test_deskewed = np.float32([HOG(deskew(im)) for im in test_images]).reshape(-1,81)

### Zmieszane dane surowe

In [15]:

train_shuffle = train_images.copy()
train_shuffle = np.float32([deskew(im) for im in train_shuffle])
np.float32([np.random.shuffle(im) for im in train_shuffle])
train_shuffle = np.asarray(train_shuffle).reshape(-1,28*28)

test_shuffle = test_images.copy()
test_shuffle = np.float32([deskew(im) for im in test_shuffle])
np.float32([np.random.shuffle(im) for im in test_shuffle])
test_shuffle = np.asarray(test_shuffle).reshape(-1,28*28)

### Dane dla uczenia sieci

In [16]:
nn_train_deskewed = np.array(hogdata_train_deskewed).reshape((60000, 81))
nn_train_deskewed = nn_train_deskewed.astype('float32') / 255

nn_test_deskewed = np.array(hogdata_test_deskewed).reshape((10000, 81))
nn_test_deskewed = nn_test_deskewed.astype('float32') / 255

nn_train_non_deskewed = np.array(hogdata_train).reshape((60000, 81))
nn_train_non_deskewed = nn_train_non_deskewed.astype('float32') / 255

nn_test_non_deskewed = np.array(hogdata_test).reshape((10000, 81))
nn_test_non_deskewed = nn_test_non_deskewed.astype('float32') / 255

nn_train_raw_deskewed = np.array(train_deskewed).reshape((60000, 28*28))
nn_train_raw_deskewed = nn_train_raw_deskewed.astype('float32') / 255

nn_test_raw_deskewed = np.array(test_deskewed).reshape((10000, 28*28))
nn_test_raw_deskewed = nn_test_raw_deskewed.astype('float32') / 255

nn_train_shuffle = np.array(train_shuffle).reshape((60000, 28*28))
nn_train_shuffle = nn_train_shuffle.astype('float32') / 255

nn_test_shuffle = np.array(test_shuffle).reshape((10000, 28*28))
nn_test_shuffle = nn_test_shuffle.astype('float32') / 255

encoded_train_labels = to_categorical(train_labels)
encoded_test_labels = to_categorical(test_labels)

### Dane dla walidacji krzyżowej

In [17]:
hogdata_train_short = hogdata_train[:600]
hogdata_train_deskewed_short = hogdata_train_deskewed[:600]

# Surowe dane 
rawdata_train_deskewed_short = train_deskewed[:600]

# Zmieszane dane surowe 
train_shuffle_short = train_shuffle[:600]

train_labels_short = train_labels[:600]

# Zadanie 2
### Powtórzyć trening klasycznych klasyfikatorów i sieci dla danych bez wyrównania (dst)

W Zadaniu 2 należy zrezygnować z zastosowania funkcji deskew do oryginalnych obrazków i porównać wyniki bez takiego preprocessingu z wynikami w notatnikach.

### Model SVM dla danych z wyrównaniem

In [18]:
model_deskewed_SVM = SVMGridSearch(hogdata_train_deskewed_short, train_labels_short)
model_deskewed_SVM.fit(hogdata_train_deskewed, train_labels)
pred_labels_deskewed_SVM = model_deskewed_SVM.predict(hogdata_test_deskewed)


Best hiperparameters:  {'C': 0.7353265306122448, 'kernel': 'linear'}


### Model Random Forest dla danych z wyrównaniem

In [19]:

model_deskewed_RF = RandomForestGridSearch(hogdata_train_deskewed_short, train_labels_short)
model_deskewed_RF.fit(hogdata_train_deskewed, train_labels)
pred_labels_deskewed_RF = model_deskewed_RF.predict(hogdata_test_deskewed)


Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   50.8s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:  2.9min finished


Best hiperparameters:  {'max_depth': 15, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 70}


### Model Neural Network dla danych z wyrównaniem

In [20]:
network_deskewed = create_model()
model_deskewed_NN = NeuralNetworkGridSrearch(nn_train_deskewed, encoded_train_labels)
network_deskewed.fit(nn_train_deskewed, encoded_train_labels, epochs=model_deskewed_NN.get('epochs'), batch_size=model_deskewed_NN.get('batch_size'))
pred_probabilities_deskewed_NN = network_deskewed.predict(nn_test_deskewed)
pred_labels_deskewed_NN = np.argmax(pred_probabilities_deskewed_NN,-1)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best hiperparameters:  {'batch_size': 30, 'epochs': 15}
Epoch 1/15
2000/2000 [==============================] - 4s 2ms/step - loss: 2.0718 - accuracy: 0.3248
Epoch 2/15
2000/2000 [==============================] - 3s 2ms/step - loss: 1.0326 - accuracy: 0.7659
Epoch 3/15
2000/2000 [==============================] - 3s 2ms/step - loss: 0.5857 - accuracy: 0.8489
Epoch 4/15
2000/2000 [==============================] - 3s 2ms/step - loss: 0.4124 - accuracy: 0.8865
Epoch 5/15
2000/2000 [==============================] - 3s 2ms/step - loss: 0.3271 - accuracy: 0.9087
Epoch 6/15
2000/2000 [==============================] - 3s 2ms/step - loss: 0.2732 - accuracy: 0.9204
Epoch 7/15
2000/2000 [==============================] - 3s 2ms/step - loss: 0.2416 - accuracy: 0.9281
Epoch 8/15
2000/2000 [==============================] - 3s 2ms/step - loss: 0.2136 - accuracy: 0.9357
Epoch 9/15
2000/2000 [==============================] - 3s 2ms/step - loss: 0.1981 - accuracy: 0.9395
Epoch 10/15
2000/2000 [===

###  Model SVM dla danych bez wyrównania

In [21]:
model_non_deskewed_SVM = SVMGridSearch(hogdata_train_short, train_labels_short)
model_non_deskewed_SVM.fit(hogdata_train, train_labels)
pred_labels_non_deskewed_SVM = model_non_deskewed_SVM.predict(hogdata_test)

Best hiperparameters:  {'C': 0.8577142857142857, 'kernel': 'linear'}


### Model RandomForest dla danych bez wyrównania

In [ ]:
model_non_deskewed_RF = RandomForestGridSearch(hogdata_train_short, train_labels_short)
model_non_deskewed_RF.fit(hogdata_train, train_labels)
pred_labels_non_deskewed_RF = model_non_deskewed_RF.predict(hogdata_test)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    9.6s


### Model Neural Network dla danych bez wyrównania

In [ ]:
network_non_deskewed = create_model()
model_non_deskewed_NN = NeuralNetworkGridSrearch(nn_train_non_deskewed, encoded_train_labels)
network_non_deskewed.fit(nn_train_non_deskewed, encoded_train_labels, epochs=model_non_deskewed_NN.get('epochs'), batch_size=model_non_deskewed_NN.get('batch_size'))
pred_probabilities_non_deskewed = network_non_deskewed.predict(nn_test_non_deskewed)
pred_labels_non_deskewed_NN = np.argmax(pred_probabilities_non_deskewed,-1)


## Wyniki

In [ ]:
#SVM wyrownane
single_result(test_labels,pred_labels_deskewed_SVM,"dane wyrownane SVM")
print("-------------------------------------------------------------")
#SVM bez wyrownania
single_result(test_labels,pred_labels_non_deskewed_SVM,"dane nie wyrownane SVM")
print("-------------------------------------------------------------")
#RandomForest wyrownane
single_result(test_labels,pred_labels_deskewed_RF,"dane wyrownane RF")
print("-------------------------------------------------------------")
#RandomForest bez wyrownania
single_result(test_labels,pred_labels_non_deskewed_RF,"dane nie wyrownane RF")
print("-------------------------------------------------------------")

#Neural Network wyrownane
single_result(test_labels,pred_labels_non_deskewed_NN,"dane nie wyrownane NN")
print("-------------------------------------------------------------")
#Neural Network bez wyrownania
single_result(test_labels,pred_labels_deskewed_NN,"dane wyrownane NN")
print("-------------------------------------------------------------")


# Wnioski
Modele w których były stosowane dane bez wyrównania (non_deskewed) były mniej dokładniejsze (Accuracy) niż te w których stosowane dane wyrównane (deskewed). Jest to związane z tym że funkcja wyrówniająca robi kształty cyfer bardziej jednolitymi i algorytmy lepiej wykrywają pobobieństwo między nimi.


# Zadanie 3
Powtórzyć trening klasycznych klasyfikatorów dla danych surowych wgranych na wejście klasyfikatora (db)

###  Model SVM dla surowych danych

In [ ]:
model_raw_deskewed = SVMGridSearch(rawdata_train_deskewed_short, train_labels_short)

In [ ]:
idx = np.random.choice(np.arange(len(train_deskewed)), 6000, replace=False)
train_deskewed_random = train_deskewed[idx]
train_labels_random = train_labels[idx]

model_raw_deskewed.fit(train_deskewed_random,train_labels_random)
pred_labels_raw_deskewed_SVM = model_raw_deskewed.predict(test_deskewed)

### Model RandomForest dla surowych danych

In [ ]:
model_raw_deskewed = RandomForestGridSearch(rawdata_train_deskewed_short, train_labels_short)
model_raw_deskewed.fit(train_deskewed, train_labels)
pred_labels_raw_deskewed_RF = model_raw_deskewed.predict(test_deskewed)

### Model Neural Network dla surowych danych

In [ ]:
network_raw_deskewed = create_model()
model_raw_deskewed = NeuralNetworkGridSrearch(nn_train_raw_deskewed, encoded_train_labels)
network_raw_deskewed.fit(nn_train_raw_deskewed, encoded_train_labels, epochs=model_raw_deskewed.get('epochs'), batch_size=model_raw_deskewed.get('batch_size'))
pred_probabilities_raw_deskewed = network_raw_deskewed.predict(nn_test_raw_deskewed)
pred_labels_raw_deskewed_NN = np.argmax(pred_probabilities_raw_deskewed,-1)


## Wyniki

In [ ]:
#SVM surowe
single_result(test_labels,pred_labels_raw_deskewed_SVM,"dane surowe SVM")
print("-------------------------------------------------------------")
#RandomForest surowe
single_result(test_labels,pred_labels_raw_deskewed_RF,"dane surowe RF")
print("-------------------------------------------------------------")
#Neural Network surowe
single_result(test_labels,pred_labels_raw_deskewed_NN,"dane surowe NN")
print("-------------------------------------------------------------")

## Wnioski
Porównując dane surowe (raw_deskewed) do danych poprzednie przetworzonych przez HOG, to można zauważyć że HOG pozwala usunąć możliwy szum w obrazkach i zmniejszych ilość informacji do przetwarazania, co powoduje o wielie szybszy czas trenowania modelu. Czas trenowania z użyciem 6000 surowych obrazków był prawie taki sam jak dla trenowania z użyciem przetworozynch  60 000 obrazków.

# Zadanie 4
Powtórzyć trening klasycznych klasyfikatorów dla zmieszanych danych surowych wgranych na wejście klasyfikatora (bdb)

### Model SVM dla zmieszanych danych surowych 

In [ ]:
model_shuffle = SVMGridSearch(train_shuffle_short, train_labels_short)
idx = np.random.choice(np.arange(len(train_shuffle)), 6000, replace=False)
train_shuffle_random = train_shuffle[idx]
train_labels_random = train_labels[idx]

model_shuffle.fit(train_shuffle_random,train_labels_random)
pred_labels_shuffle_SVM = model_shuffle.predict(test_shuffle)


### Model Random Forest dla zmieszanych danych surowych 

In [ ]:
model_shuffle = RandomForestGridSearch(train_shuffle_short, train_labels_short)
model_shuffle.fit(train_shuffle, train_labels)
pred_labels_shuffle_RF = model_shuffle.predict(test_shuffle)

### Model Neural Network dla zmieszanych danych surowych 

In [ ]:
network_shuffle = create_model()
model_shuffle = NeuralNetworkGridSrearch(nn_train_shuffle, encoded_train_labels)
network_shuffle.fit(nn_train_shuffle,encoded_train_labels, epochs=model_shuffle.get('epochs'), batch_size=model_shuffle.get('batch_size'))
pred_probabilities_shuffle = network_shuffle.predict(nn_test_shuffle)
pred_labels_shuffle_NN = np.argmax(pred_probabilities_shuffle,-1)


In [ ]:
#SVM mieszane
single_result(test_labels,pred_labels_shuffle_SVM,"dane mieszane SVM")
print("-------------------------------------------------------------")
#RandomForest mieszane
single_result(test_labels,pred_labels_shuffle_RF,"dane mieszane RF")
print("-------------------------------------------------------------")
#Neural Network mieszane
single_result(test_labels,pred_labels_shuffle_NN,"dane mieszane NN")
print("-------------------------------------------------------------")

# Wnioski
Porównując wyniki danych mieszanych surowych i surowych danych widzimy że dokładnośc znacząca zwiększyła się, co pokazuję że HOG naprawdę ulepsza dokładność oraz czas obliczeń.